## Libraries

In [1]:
from sklearn.preprocessing import StandardScaler
import librosa
from IPython.display import clear_output
import IPython.display as ipd  # To play sound in the notebook
import IPython

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils import data
import torch.nn as nn

import plotly.graph_objs as go
import os
import math
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

# Standard plotly imports
#import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objs as go
init_notebook_mode(connected=True)
#import plotly.plotly as py

# Using cufflinks in offline mode
#import cufflinks
# cufflinks.go_offline()

# Set the global theme for cufflinks
#cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)


#from sklearn.model_selection import train_test_split

# external .py with functions used for processing
#from functionsdef import *

# import scipy.io.wavfile as sci_wav  # Open wav files
#import scipy
#import json, codecs

# import warnings  # Warning removal
# warnings.filterwarnings('ignore')

# automaitc reload modules that have changed
%load_ext autoreload
%autoreload 2

%autosave 60

# Enable 2x images for better resolution in retina displays
%config InlineBackend.figure_format = 'retina'

Autosaving every 60 seconds


## Global parameters

In [2]:
#Sampling rate
sr = int(16e3)
#FFT bins
n_fft = 512
#window size in msec. Note that the final value in samples will be the next power of two

w = 20e-3
#Window size in samples
win_length = int(w*sr)
#Next power of two (samples)
win_length = 1<<(win_length-1).bit_length()

#Hop size in msec. Note that the final value in samples will be the next power of two.
h = 10e-3
#Hop size in samples
hop_length = int(h*sr)
#Next power of two
hop_length = 1<<(hop_length-1).bit_length()


#Number of neighbor frames to form the input feature vectors
neighbour_frames = 5
#Lenght of the feature vector. This is the rows of the STFT * (rev current frame + rev neighbor frames+anechoic frame)
input_length = int( (n_fft/2 + 1) * (neighbour_frames + 1) )
output_length = int(n_fft/2 + 1)
feature_length = input_length + output_length

anechoic_dir = 'anechoic/'
reverberant_dir = 'reverberant/'



## Data exploration

In [ ]:
#Search for .wav files in the given directories and build the filenames list
#anechoic_filenames = [_ for _ in os.listdir(anechoic_dir) if _.endswith(".wav")]
#reverberant_filenames = [_ for _ in os.listdir(reverberant_dir) if _.endswith(".wav")]
anechoic_filenames = [_ for _ in os.listdir(anechoic_dir) if _.endswith(".wav")]
reverberant_filenames = [_ for _ in os.listdir(reverberant_dir) if _.endswith(".wav")]

print("There are {} anechoic files".format(len(anechoic_filenames)))
print("There are {} reverberant files".format(len(reverberant_filenames)))
print("The mapping from reverberant to anechoic has a ratio of", len(reverberant_filenames)/len(anechoic_filenames))

In [ ]:
random=np.random.randint(0,len(anechoic_filenames))

In [ ]:
print("Random Anechoic file")
ipd.Audio(anechoic_dir + anechoic_filenames[random])

In [ ]:
print("Random reverberant file")
ipd.Audio(reverberant_dir + reverberant_filenames[random])

In [ ]:
anecdur = {}
for idx, val in enumerate(anechoic_filenames):
    clear_output(wait=True)
    timeseries,_ = librosa.load(anechoic_dir+val,sr=sr)
    dur = librosa.get_duration(timeseries, sr=sr)
    data = {'filename': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}
    anecdur[val] = dur
    print('Current progress', np.round(idx/len(anechoic_filenames)*100, 2),'%')

In [ ]:
dfanec = pd.DataFrame(list(anecdur.items()), columns=['Filename', 'Duration(seg)'])
dfanec.head()

In [ ]:
revdur = {}
for idx, val in enumerate(reverberant_filenames):
    clear_output(wait=True)
    timeseries,_ = librosa.load(reverberant_dir+val,sr=sr)
    dur = librosa.get_duration(timeseries, sr=sr)
    revdur[val] = dur
    print('Current progress', np.round(idx/len(reverberant_filenames)*100, 2),'%')

In [ ]:
dfrev = pd.DataFrame(list(revdur.items()), columns=['Filename', 'Duration(seg)'])
dfrev.head()

Plot

In [ ]:
x1 = dfanec['Duration(seg)'].values.tolist()
x2 = dfrev['Duration(seg)'].values.tolist()

#histnorm='probability',
trace1 = go.Histogram(x=x1, name='Anechoic audios', opacity=0.75, xbins=dict(
    start=np.min(x1), size=0.2, end=np.max(x1)))

trace2 = go.Histogram(x=x2, name='Reverberant audios', opacity=0.75, xbins=dict(
    start=np.min(x2), size=0.2, end=np.max(x2)))

layout = go.Layout(xaxis=dict(title='Duration(seg)'), title="Histogram with frequency count", barmode='overlay')

data = [trace1, trace2]

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='histogram-discrete-freq-count')

In [ ]:
trace3 = go.Histogram(x=x1, cumulative=dict(enabled=True), name='Anechoic audios', opacity=0.75,)
trace4 = go.Histogram(x=x2, cumulative=dict(enabled=True), name='Reverberant audios', opacity=0.75)

layout = go.Layout(xaxis=dict(title='Duration(seg)'), title="Histogram with Cumulative", barmode='overlay')

data = [trace3, trace4]

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='histogram-discrete-freq-count')

## Processing functions

In [3]:
def rescale(x, flatten, min, max):
    """Scales data to the range given by [min, max]. If x is a matix it scales in Axis 0 
    (i.e. for 2D arrays running vertically downwards across rows)
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html 
    x: numpy array.
    flatten: boolean. If true, it flattens x to 1D, then reshapes back.
    min: float. Minimum number to scale data to. 
    max: float. Maximum number to scale data to. 
    """

    if flatten:
        if x.ndim > 1:
            r, c = np.shape(x)
            x = x.flatten()
            # Neccesary because flatten removes dimension and StandardScaler() requires 2dim
            x = x.reshape(-1, 1)
            scaler = MinMaxScaler(feature_range=(min, max))
            xrescaled = scaler.fit_transform(x)
            #xrescaled = np.reshape(xrescaled, (r, c))
            xrescaled = xrescaled.reshape(r, c)
            return xrescaled
        else:
            # Necessary because StandardScaler() requires 2dim
            x = x.reshape(-1, 1)
            r, c = np.shape(x)
            scaler = MinMaxScaler(feature_range=(min, max))
            xrescaled = scaler.fit_transform(x)
            xrescaled = xrescaled.reshape(r, )
            return xrescaled

    else:
        if x.ndim > 1:
            scaler = MinMaxScaler(feature_range=(min, max))
            xrescaled = scaler.fit_transform(x)
            return xrescaled
        else:
            # Necessary because StandardScaler() requires 2dim
            x = x.reshape(-1, 1)
            r, c = np.shape(x)
            scaler = MinMaxScaler(feature_range=(min, max))
            xrescaled = scaler.fit_transform(x)
            xrescaled = xrescaled.reshape(r, )
            return xrescaled


def STFTanalysis(x, type_return, dtype='complex64'):
    """Calculates the Short-Time Fourier transform.
    x: numpy array
    type_return: String. Type of STFT. 'complex', 'mag' or 'phase'. 
    dtype: Librosa defaults to complex64 when the complex STFT is calculated. If 'mag' is used as type_return, 
    dtype will be float32. When using 'complex' or 'phase' dtype will be complex64.
    Torch can only convert to tensor dtypes of type double, float, float16, int64, int32, and uint8. 

    https://librosa.github.io/librosa/generated/librosa.core.stft.html
    https://librosa.github.io/librosa/generated/librosa.core.magphase.html
    """

    STFT = librosa.stft(x, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window='hamming',
                        center=True, pad_mode='reflect', dtype=dtype)

    if type_return == 'complex':
        return STFT
    elif type_return == 'mag':
        magSTFT, _ = librosa.magphase(STFT)
        return magSTFT
    elif type_return == 'phase':
        _, phaseSTFT = librosa.magphase(STFT)
        # Phase angle in radians
        return np.angle(phaseSTFT)


def standardScale(x, Transpose=False):
    """Standardize features by removing the mean and scaling to unit variance. If x is a 
    matrix it scales across the first axis (i.e. for 2D arrays running vertically 
    downwards across rows). For frequency bins, it should be left as default. 
    Assumes Gaussian distribution. Data ends with a mean of 0 and a standard deviation of 1.
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
    directory.
    x: numpy array
    Transpose: Boolean. If True, the input matrix is transposed prior to transformation.
    """
    scaler = StandardScaler().fit(x)
    xstandard = scaler.transform(x)
    return xstandard


def powerdB(x):
    """Convert a power spectrogram (amplitude squared) to decibel (dB) units
    x: spectrogram
    https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
    """
    # x_dB = 10 * log10(x**2) = 20 log10(x)
    # return librosa.amplitude_to_db(x)
    return librosa.power_to_db(x**2)


def crop(reverberant, anec_len):
    """Crop reverberant audios, in such a way that its length is equal to:
    [ length of its corresponding anechoic  + (number_neighbour_frames * win_length) ]. 
    This is performed in order to ensure the same length of anechoic and reverberant STFT
    which will conform the feature pairs to feed the NN
    Note that IF the filenames of the anechoic and reverberant are not related, the function
    will not work. The files used here have the following syntax to ensure an easy association of
    the anechoic file that gave birth to the reverberant version:
    anechoic:       ABCD_##.wav
    reverberant:    ABCD_##-rir-#.#-r#.wav 
    The first 7 characters always correlate thus allowing to map the reverberant with its corresponding
    anechoic file.
    reverberant: STFT matrix.
    anec_len: Length of the anechoic audio file that corresponds to the reverberant audio file
    """
    reverberant_cropped = reverberant[:, 0: anec_len + neighbour_frames]
    return reverberant_cropped


def featureMatrix(reverberant, anec_col):
    '''
    Matrix of the feature vectors (frame + neighbour_frames). 
    reverberant_col_n + reverberant_col_n+1 + ... + reverberant_col_n+5
    For the case of STFT with a size of 512, the resulting matrix has a shape of [ N x 1542] 
    where N depends on the duration of the audiofile. In summary, each row now is the input vector to the NN.

    '''

    for k in range(anec_col):
        fvector = reverberant[:, k:k+1+neighbour_frames]
        fvector = np.reshape(fvector, (1, -1), order='F')
        if k == 0:
            fmatrix = fvector
            # print(k)
            # print(fmatrix.shape)
            # print(fmatrix)
        else:
            fmatrix = np.concatenate((fmatrix, fvector), axis=0)
            # print(k)
            # print(fmatrix.shape)
            # print(fmatrix)

    return fmatrix


def random_idx():
    # Function to get random indices for split. For future use
    # import random
    # from math import floor

    # def train_valid_split(dataset, test_size = 0.25, shuffle = False, random_seed = 0):
    #     """ Return a list of splitted indices from a DataSet.
    #     Indices can be used with DataLoader to build a train and validation set.

    #     Arguments:
    #         A Dataset
    #         A test_size, as a float between 0 and 1 (percentage split) or as an int (fixed number split)
    #         Shuffling True or False
    #         Random seed
    #     """
    #     length = dataset.__len__()
    #     indices = list(range(1,length))

    #     if shuffle == True:
    #         random.seed(random_seed)
    #         random.shuffle(indices)

    #     if type(test_size) is float:
    #         split = floor(test_size * length)
    #     elif type(test_size) is int:
    #         split = test_size
    #     else:
    #         raise ValueError('%s should be an int or a float' % str)
    # return indices[split:], indices[:split]
    return

## Model

In [4]:
#From https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class Dereverbation(Dataset):
    
    def __init__(self, anecdir, revdir):
        """
        Args:
            anecdir (string): Path to the reverberant wav file. Tipically = 'anechoic/'
            revdir (string): Path to the reverberant wav file. Tipically = 'reverberant/'
        
        The class makes use of the fact that the filenames of the reverberant and anechoic are related following 
        the rule that the ith anechoic has a reverberant file with the same name plus other information. E.g.:
        CA01_01.wav
        CA01_01-rir-0.3-r1.wav
        So by getting the first 7 characters + 'wav' from the reverberant file, we can get its corresponding 
        anechoic version
        """
        self.anecdir = anecdir
        self.revdir = revdir
        #self.anechoic_filenames = [_ for _ in os.listdir(anecdir) if _.endswith(".wav")]
        self.reverberant_filenames = [_ for _ in os.listdir(revdir) if _.endswith(".wav")]
        
        #Dicttionary [reverberant:Anechoic] = [input:output]
        self.dictfiles = { i : i[0:7]+'.wav' for i in self.reverberant_filenames }
        
        #the same with for loop
        #for j in range(len(self.reverberant_filenames)):
            #self.dictfiles[self.reverberant_filenames[j]] = self.reverberant_filenames[j][0:7] + '.wav'
    
    def __len__(self):
        return len(self.dictfiles)

    def __getitem__(self, idx):
        """
        Returns a tuple of 3 elements:
        1. A dict with input/output ('reverberant', 'anechoic'). 
        Each matrix row from 'reverberant' is the input.
        Each matrix row from 'anechoic' is the output.
        2. Reverberant filename (rev_fnm).
        3. Anechoic filename (anec_fnm).
        
        e.g. [index][0] --> dict
        e.g. [index][0]['reverberant'] --> input
        e.g. [index][0][1] --> reverberant filename
        """
        reverberant_fnm = self.reverberant_filenames[idx]
        anechoic_fnm = self.dictfiles[self.reverberant_filenames[idx]]
        
        
        #Feature processing
        anechoic, _ = librosa.load(self.anecdir + anechoic_fnm, sr=sr)
        anechoic = np.trim_zeros(anechoic, 'f')
        anechoic = STFTanalysis(anechoic,'mag')
        anechoic = standardScale(anechoic)
        anechoic = powerdB(anechoic)
        anec_rows, anec_col = np.shape(anechoic)
        anechoic = np.transpose(anechoic)
        
        reverberant, _ = librosa.load(self.revdir + reverberant_fnm, sr=sr)
        reverberant = np.trim_zeros(reverberant, 'f')
        reverberant = STFTanalysis(reverberant,'mag')
        reverberant = standardScale(reverberant)
        reverberant = powerdB(reverberant)
        reverberant = crop(reverberant, anec_col)
        reverberant = featureMatrix(reverberant, anec_col)
        
        sample = {'reverberant': torch.from_numpy(reverberant), 'anechoic': torch.from_numpy(anechoic)} 
        return sample, reverberant_fnm, anechoic_fnm

    

In [5]:
#Dataset class instance
transformed_dataset = Dereverbation('anechoic/','reverberant/')

In [7]:
#Create split for train/eval.

eval_perc = 0.2
n = len(transformed_dataset)  # number of elements in the dataset
n_eval = int(n * eval_perc)  # number of eval elements
n_train = n - n_eval   #number of training elements
idx = list(range(n))  # indices to all elements
np.random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
train_idx = idx[:n_train] #train index numbers
eval_idx = idx[n_train:]  #eval index numbers
#len(train_idx)

train_set = data.Subset(transformed_dataset, train_idx)
eval_set = data.Subset(transformed_dataset, eval_idx)
#Equivalent
#train_set, eval_set = data.random_split(transformed_dataset, (n_train, n_eval))

In [203]:
#example of train files
for i in range(n_train):
    sample = train_set[i]
    print(i, 'input:', sample[0]['reverberant'].shape, sample[1], ' / ', 
          'output:', sample[0]['anechoic'].shape, sample[2])
    if i == 3:
        break

0 input: torch.Size([127, 1542]) FG37_03-rir-0.9-r2.wav  /  output: torch.Size([127, 257]) FG37_03.wav
1 input: torch.Size([156, 1542]) MH44_09-rir-0.6-r2.wav  /  output: torch.Size([156, 257]) MH44_09.wav
2 input: torch.Size([163, 1542]) FL69_05-rir-0.9-r2.wav  /  output: torch.Size([163, 257]) FL69_05.wav
3 input: torch.Size([136, 1542]) MC14_05-rir-0.9-r2.wav  /  output: torch.Size([136, 257]) MC14_05.wav


In [261]:
#NN params
input_size = 1542
hidden_size = 500
num_classes = 257
num_epochs = 1
batch_size = 1 #fix because it is only working when equal to 1
learning_rate = 0.001
num_workers = 4


dataloader_train = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
dataloader_eval = DataLoader(dataset=eval_set, batch_size=batch_size, shuffle=True)

In [262]:
#e.g.
for index, (dict, revfnm, anecfnm) in enumerate(dataloader_eval):
    
    print(index, 'input:', dict['reverberant'].size(), revfnm, ' > ', 
          'output:', dict['anechoic'].size(), anecfnm)
    if index == 3:
        aa = dict['reverberant'].view(-1, input_size)
        print(aa.shape)
        break

0 input: torch.Size([1, 132, 1542]) ('FB11_08-rir-0.9-r2.wav',)  >  output: torch.Size([1, 132, 257]) ('FB11_08.wav',)
1 input: torch.Size([1, 121, 1542]) ('MJ60_02-rir-0.6-r1.wav',)  >  output: torch.Size([1, 121, 257]) ('MJ60_02.wav',)
2 input: torch.Size([1, 132, 1542]) ('FF36_02-rir-0.9-r2.wav',)  >  output: torch.Size([1, 132, 257]) ('FF36_02.wav',)
3 input: torch.Size([1, 150, 1542]) ('FE27_09-rir-0.9-r1.wav',)  >  output: torch.Size([1, 150, 257]) ('FE27_09.wav',)
torch.Size([150, 1542])


In [263]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [265]:

# Train the model
total_step = len(dataloader_train)
for epoch in range(num_epochs):
    for index, (dict, revfnm, anecfnm) in enumerate(dataloader_train):
        #cold be anechoic, as they have the same rows
        #print(row)
        for row in range(dict['reverberant'].size()[1]):
            #Move tensors to the configured device
            #print(dict['reverberant'][:,row,:].shape)
            #print(dict['anechoic'][:,row,:].shape)
            images = dict['reverberant'][:,row,:].to(device)
            labels = dict['anechoic'][:,row,:].to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (index+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, index+1, total_step, 
                                                                      loss.item()))

Epoch [1/1], Step [100/6932], Loss: 3.3170
Epoch [1/1], Step [200/6932], Loss: 6.1051
Epoch [1/1], Step [300/6932], Loss: 8.9047
Epoch [1/1], Step [400/6932], Loss: 4.6612
Epoch [1/1], Step [500/6932], Loss: 6.0712
Epoch [1/1], Step [600/6932], Loss: 3.1502
Epoch [1/1], Step [700/6932], Loss: 2.2776
Epoch [1/1], Step [800/6932], Loss: 5.1448
Epoch [1/1], Step [900/6932], Loss: 5.2633
Epoch [1/1], Step [1000/6932], Loss: 3.9897
Epoch [1/1], Step [1100/6932], Loss: 4.1940
Epoch [1/1], Step [1200/6932], Loss: 5.5710
Epoch [1/1], Step [1300/6932], Loss: 4.0104
Epoch [1/1], Step [1400/6932], Loss: 1.8064
Epoch [1/1], Step [1500/6932], Loss: 2.6658
Epoch [1/1], Step [1600/6932], Loss: 5.9620
Epoch [1/1], Step [1700/6932], Loss: 3.2618
Epoch [1/1], Step [1800/6932], Loss: 4.8116
Epoch [1/1], Step [1900/6932], Loss: 15.2244
Epoch [1/1], Step [2000/6932], Loss: 3.7350
Epoch [1/1], Step [2100/6932], Loss: 7.9603
Epoch [1/1], Step [2200/6932], Loss: 5.2371
Epoch [1/1], Step [2300/6932], Loss: 2.6

In [11]:
# Loop over epochs
for epoch in range(max_epochs):
    # Training
    for local_batch, local_labels in training_generator:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)

        # Model computations
        [...]

    # Validation
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in validation_generator:
            # Transfer to GPU
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)

            # Model computations
            [...]




NameError: name 'training_generator' is not defined